In [11]:
# 1.Daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

df=pd.read_excel(r'./DataSource/DailyUpdateForTG.xlsx',skiprows=1,sheet_name=1)
df['城市']=df['指标名称'].str.split(":").str[-1]
df=df.drop(columns=['指标名称','单位','来源','更新时间'])
df.to_hdf('Processed_Data/subway.h5',key='raw')

subway=pd.read_hdf('Processed_Data/subway.h5',key='raw')
subway=pd.melt(subway,id_vars=['城市'],var_name='date',value_name='subway')
subway=subway[subway['subway']>0]
subway.head(3)

,城市,date,subway
1,上海,2022-12-28,395.10
2,广州,2022-12-28,468.00
4,成都,2022-12-28,488.08


In [13]:
#TG update

list_tokeep='上海,深圳,广州,成都,西安,北京,苏州,重庆,武汉,郑州,南京'.split(',') #南京
s1=subway['城市'].isin(list_tokeep)
df=subway[s1].groupby(by='date').sum().reset_index()
df2=subway[s1].groupby(by='date').count().reset_index()
df['subway_7dbefore']=df['subway'].shift(periods=7)
df['subway_wow']=df['subway']/df['subway'].shift(periods=7) -1

df['date']=pd.to_datetime(df['date'])

df['weekday']=df['date'].dt.strftime('%A')

df['workday']=np.nan
df['weekend']=np.nan

c1=df['weekday'].isin(['Saturday','Sunday'])

df.loc[~c1,'workday']=df['subway']
df.loc[c1,'weekend']=df['subway']

df['sum']=np.nan

df=df[['date','sum','subway_wow','weekend','workday']]

df=df.sort_values(by='date')
df.tail(31).to_clipboard(index=False,header=None)
print(df.tail(10))
print(df2.tail(5))

           date  sum  subway_wow  weekend  workday
1971 2022-12-19  NaN   -0.382490      NaN  2223.08
1972 2022-12-20  NaN   -0.377058      NaN  2043.35
1973 2022-12-21  NaN   -0.326942      NaN  2019.68
1974 2022-12-22  NaN   -0.312817      NaN  1880.07
1975 2022-12-23  NaN   -0.165496      NaN  2096.75
1976 2022-12-24  NaN    0.108071  1592.83      NaN
1977 2022-12-25  NaN    0.220820  1537.66      NaN
1978 2022-12-26  NaN    0.224508      NaN  2722.18
1979 2022-12-27  NaN    0.366550      NaN  2792.34
1980 2022-12-28  NaN   -0.056786      NaN  1904.99
           date  城市  subway
1976 2022-12-24  11      11
1977 2022-12-25  11      11
1978 2022-12-26  11      11
1979 2022-12-27  11      11
1980 2022-12-28   7       7


In [25]:
df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
df_covid.tail(2)

,city,progress,peak,end,城市,常住人口,progress_2
295,聊城,回落至34%,2022/12/20,2023/03/07,聊城,201,34
296,宿迁,回落至41%,2022/12/21,2023/03/08,宿迁,310,41


In [28]:
# TY 地铁数据 vs. 2019
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
riqi='2022-12-26'

subway=pd.read_hdf('Processed_Data/subway.h5',key='raw')
subway=pd.melt(subway,id_vars=['城市'],var_name='date',value_name='subway')

subway=subway[subway['subway']>0]

subway['date'] = pd.to_datetime(subway['date']).dt.date
subway=subway.merge(df_covid,on=['城市'],how='left')
subway=subway.sort_values(by=['城市','date'],ascending=True)
subway['date']=subway['date'].astype('str')


def getdt(d):
    return str(d)[-5:]
subway['dt']=subway['date'].apply(getdt)

subway['subway_last7d_mean']=subway.groupby('城市')['subway'].transform(lambda x: x.rolling(7, 3).mean())
subway['subway_WoW']=subway['subway_last7d_mean']/subway.groupby('城市')['subway_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=subway['date'].str.contains('2019')
sw_19=subway[c1]
sw_19=sw_19[['城市', 'dt','subway_last7d_mean']]
sw_19=sw_19.rename(columns={'subway_last7d_mean':'subway_19_rolling_mean'})

c2=subway['date'].str.contains('2022')
sw_22=subway[c2]
sw_22=sw_22.merge(sw_19,how='left',on=['城市','dt'])

sw_22['subway_status']=sw_22['subway_last7d_mean']/sw_22['subway_19_rolling_mean']

sw_22['covid_rank']=sw_22.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)
o=sw_22[sw_22['subway_status']<2][sw_22['date']==riqi][['城市','covid_rank','subway_status']] 

o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)
print(riqi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-12-26


In [19]:
sw_22

,城市,date,subway,city,progress,peak,end,常住人口,progress_2,dt,subway_last7d_mean,subway_WoW,subway_19_rolling_mean,subway_status,covid_rank
0,上海,2022-01-01,724.90,上海,回落至68%,2022/12/22,2023/01/29,2488,68,01-01,1025.257143,0.004043,995.114286,1.030291,2.0
1,上海,2022-01-02,725.00,上海,回落至68%,2022/12/22,2023/01/29,2488,68,01-02,1042.085714,0.023387,996.342857,1.045911,2.0
2,上海,2022-01-03,703.90,上海,回落至68%,2022/12/22,2023/01/29,2488,68,01-03,983.900000,-0.031377,995.657143,0.988192,2.0
3,上海,2022-01-04,1164.90,上海,回落至68%,2022/12/22,2023/01/29,2488,68,01-04,989.714286,-0.025282,995.228571,0.994459,2.0
4,上海,2022-01-05,1105.70,上海,回落至68%,2022/12/22,2023/01/29,2488,68,01-05,985.228571,-0.027703,938.671429,1.049599,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,长沙,2022-12-17,67.01,长沙,回落至40%,2022/12/21,2023/01/22,599,40,12-17,119.015714,-0.073983,102.330000,1.163058,8.0
9809,长沙,2022-12-18,54.80,长沙,回落至40%,2022/12/21,2023/01/22,599,40,12-18,110.558571,-0.179415,102.345714,1.080246,8.0
9810,长沙,2022-12-19,72.04,长沙,回落至40%,2022/12/21,2023/01/22,599,40,12-19,99.310000,-0.280675,101.094286,0.982350,8.0
9811,长沙,2022-12-20,63.02,长沙,回落至40%,2022/12/21,2023/01/22,599,40,12-20,88.054286,-0.368819,100.120000,0.879487,8.0


In [32]:
citylist=['北京','成都','重庆','武汉','上海']
c3=sw_22['城市'].isin(citylist)
c4=sw_22['date']>'2022-03-01'
show=sw_22[c3&c4]


show=pd.pivot_table(show,index='date',columns='城市',values='subway_status',aggfunc='mean').reset_index()
show['sum']=np.nan

show=show[['date','sum']+citylist]
show.to_clipboard(index=False)

In [30]:
# TY 地铁数据 WoW
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

sw_22['covid_rank']=sw_22.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)
o=sw_22[sw_22['date']==riqi][['城市','covid_rank','subway_WoW']] 

o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)
print(riqi)

2022-12-26


In [33]:
#TY copy to wkbk

head=pd.read_clipboard()
wkbk=pd.pivot_table(subway[subway['date']>'2022-12-21'],index='城市',columns='date',values='subway',aggfunc='mean').reset_index()
head.merge(wkbk,how='left',on='城市').to_clipboard(index=False)